In [ ]:
import gensim
model = gensim.models.Word2Vec.load('word2vec_combine_trained')

In [ ]:
import jieba

jieba.set_dictionary('traditional.dict.txt')
jieba.load_userdict('additional.dict.txt')

In [ ]:
import json

def load_testing_data(data):
    f = open(data, 'r')
    questions = f.readlines()
    q = list()
    for question in questions:
        try:
            js =json.loads(question)
            q.append(js)
        except:
            print question
    f.close()
    return q #q is a list


In [ ]:
f1 = open('questions.txt', 'r')
q_list = f1.readlines()

In [ ]:
f = open('Pix_sentence', 'r')
s_list = f.readlines()

In [ ]:
def bin_search(i, a_list):
    if len(i) <= 12:
        return 'DNE'
    
    min_v = 0
    max_v = len(a_list)
    mid = 0
    while(mid < max_v):
        mid = (min_v + max_v)/2
        
        if i in a_list[mid]: 
            return a_list[mid]

        elif i > a_list[mid]:
            min_v = mid+1
    
        else:
            max_v = mid
        mid = (min_v + max_v)/2
    
    return 'DNE'


In [ ]:
def print_list(ans):
    w = ''
    for i in ans:
        w += i + ' '
    print w

In [ ]:
stopWords_set = set()
content = open('ch_stopwords.txt', 'rb').read().decode('utf-8')
for line in content.splitlines():
    stopWords_set.add(line.strip().encode('utf8'))
    stopWords_set.add(' ')

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text
    
#this function returns the paragraph as a list of words(without the buzzword)
def get_cloze_par(par): 

    #par is the paragraph without any punctuation.
    reps = {'。':'', '!':'', ',':'', '.':'', '，':'', '～': '', '~':'', '︽⊙＿⊙︽':'BUZZWORD'}
    par = replace_all(par, reps)
    
    words = jieba.cut(par, cut_all=False)
    ans = list()
    for w in words:
        w = w.encode('UTF-8')
        if w not in stopWords_set and w in model:
            ans.append(w)
    return ans

In [ ]:
#get the highest score from the paragraph +1/10 of the entire paragraph
def relation_score(a):
    s = a[0]
    g = 0
    for i in range(len(a)):
        if a[i] > s:
            s = a[i]
        g += a[i]
    s += g/10.
    return s

def get_answer(nums):
    ans = 0
    temp = nums[0]
    for i in range(len(nums)):
        if(nums[i] > temp):
            ans = i
            temp = nums[i]
    return ans

In [ ]:
def ans_score(nums):
    temp = nums[0]
    for i in range(len(nums)):
        if(nums[i] > temp):
            temp = nums[i]
    return temp

In [ ]:
def check_answer(ans_list, correct_ans, score_list):
    o = 0;
    zero = 0
    zero_c = 0
    one = 0
    one_c = 0
    two = 0
    two_c = 0
    three = 0
    three_c = 0
    score = 0
    r_list = ['']*30
    wrong_list = ['']*30
    c_score = 0
    w_score = 0
    for y in range(30):
        if(ans_list[y] == correct_ans[y]):
            c_score += score_list[y]
            o +=1
            duu = 'correct'
            if(guess_list[y] == ''):
                zero +=1
                zero_c +=1
            elif(guess_list[y] == '*'):
                one +=1
                one_c +=1
            elif(guess_list[y] == '**'):
                two +=1
                two_c +=1
            elif(guess_list[y] == '***'):
                three +=1
                three_c +=1
        else:
            duu = 'wrong'
            w_score += score_list[y]
            wrong_list[y] += '\t' + str(ans_list[y]) + ' answer: ' + str(correct_ans[y])
            if(guess_list[y] == ''):
                zero +=1
            elif(guess_list[y] == '*'):
                one +=1
            elif(guess_list[y] == '**'):
                two +=1
            elif(guess_list[y] == '***'):
                three +=1

        r_list[y] += duu
        score += score_list[y]

    print 'Correctness:'
    print o/30. 
    score = score/30.
    print "Average score: " + str(score)
    print 'correct average score: ' + str(c_score/o)
    print 'wrong_ans average score: ' + str(w_score/(30-o))
    print '\'\': ' + str(zero_c/float(zero))
    print '*: ' + str(one_c/float(one))
    print '**: ' + str(two_c/float(two))
    #print '***: ' + str(three_c/float(three))
    print '\n'
    
    for y in range(30):
        print str(y+1) + ': ' + r_list[y] + '\t' + str(score_list[y]) + '\t'+ guess_list[y] + '\t' + w_list[y] + '\t' + wrong_list[y]
        score += score_list[y]
    

In [ ]:
def search_term(par):
    reps = {'。':'\n', '!':'\n', ',':'\n', '.':'\n', '，':'\n', '～': '/n', '~':'\n', '︽⊙＿⊙︽':'BUZZWORD'}
    par = replace_all(par, reps)
    sentences = par.splitlines()
    term = ''
    sentence = ""
    for i in range(len(sentences)):
        if 'BUZZWORD' in sentences[i]:
            sentence = sentences[i].strip()
    
    words = jieba.cut(sentence, cut_all=False)
    for w in words:
        w = w.strip()
        w = w.encode('UTF-8')
        if w == 'BUZZWORD':
            if len(term) == 0:
                return "DNE"
            return term
        else:
            term = term + w + ' '
    
    return term

In [ ]:
def relation_score2(a, b):
    s = a[0]
    g = 0
    for i in range(len(a)):
        if a[i] > s:
            s = a[i]
        g += a[i]
    s += g/10. + b
    return s

In [ ]:
def question2json(file):
    f1 = open(file, 'r')
    tq = f1.readlines()


    for line in tq:
        result = '{\"answer\": \"\", '
        pattern = re.compile('.*\[(\d+)\](.*)###(.*)\[END\]', re.DOTALL)
        if pattern.match(line):
            m = re.match('.*\[(\d+)\](.*)###(.*)\[END\]', line, re.DOTALL)
            quiz_no = int(m.group(1).strip())
            question = m.group(2)
            result += '\"choices\": {'
            for item in m.group(3).split(','):
                index, value = item.split(':')
                result += '\"' + index.strip() + '\": \"' + value.strip() +'\", '
            result = result[:-2]
            result += '}, \"index\": ' + str(quiz_no) + ", \"question\": \"" + question +'\"}'
            print result
        
def test_bot(q_list):
    i = 0
    for line in q_list:
        if '機器人小朋友請搶答' in line:
            hello_send(None) 
        else:
            i += 1
            print i
            receive_question(None, line)

In [ ]:
import time
start_time = time.time()

data = load_testing_data('JSon_question.txt')
correct_ans = []
ans_list = []
guess_list = ['']*len(data)
w_list = ['']*len(data)
score_list = []

for i in range(len(data)):
    
    f1 = data[i]['question'].encode('UTF-8')
    correct_ans.append(data[i]['answer'])
    choices = ['a', 'b', 'c', 'd', 'e']
    pot = []
    final_ans = ''
    #search method
    search_ = search_term(f1).strip()
    search_w = search_.split()
    #list of words
    ans_ = bin_search(search_, s_list)

    if ans_ == 'DNE':
        ans = 'DNE'
    else:
        ans = ans_.split()[len(search_w)]
      
    for j in choices:
        c = data[i]['choices'][j].encode('UTF-8')
        if c.lower() == ans.lower():
            final_ans = j
            break
    if final_ans != '':
        ans_list.append(final_ans)
        score_list.append(100)
        
    else:
        words = get_cloze_par(f1)
        for j in choices:
            c = data[i]['choices'][j].encode('UTF-8')
            #假如選項不在 model 裡 用結巴斷句
            if c not in model: 
                w_list[i] += c + '  '
                guess_list[i] += '*'
                temp = jieba.lcut(c, cut_all=False)
                c = temp[0].encode('UTF-8')
                #假如斷完之後還是不在 model 中, 取第一個字
                if c not in model:
                    c = c[0:3]
                    guess_list[i] += '*'
                #英文字的話直接代換
                if c not in model:
                    c = '嗨'
                    guess_list[i] += '*'
                w_list[i] += c 
            if ans == 'DNE':
                add_ = 0
            else:
                add_ = model.similarity(ans, c)
            b = model.similarity(words, c)
            pot.append(relation_score2(b, add_))

        ans_list.append(choices[get_answer(pot)])
        score_list.append(ans_score(pot))    

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
check_answer(ans_list, correct_ans, score_list)